# Setup

In [1]:
run manifold_twins

In [2]:
a = ManifoldTwinsAnalysis()

In [3]:
a.run_analysis()

Loading dataset...
    IDR:          BLACKSTON
    Phase range: [-5.0, 5.0] days
    Bin velocity: 1000.0


100%|██████████| 415/415 [00:18<00:00, 22.13it/s]


Estimating the spectra at maximum light...
    Loaded cached stan model
    Using saved stan result
Reading between the lines...
    Loaded cached stan model
    Using saved stan result
Building masks...
    Masking 30/203 targets whose interpolation uncertainty power is 
    more than 0.100 of the intrinsic power.
Generating the manifold learning embedding...
Calculating spectral indicators...
Fitting GP hyperparameters...
    Fit result:           Optimization terminated successfully.
    Color scale:          -0.007 ± 0.070
    Intrinsic dispersion: 0.065 ± 0.013 mag
    GP kernel amplitude:  0.111 ± 0.042 mag
    GP length scale:      3.348 ± 2.272
    Fit NMAD:             0.072 mag
    Fit std:              0.098 mag
Calculating SALT2 Hubble residuals...
SALT2 Hubble fit: 
    ref_mag: -10.449
    alpha:   0.142
    beta:    2.665
    σ_int:   0.134
    RMS:     0.156
    NMAD:    0.110
    WRMS:    0.156
Loading host galaxy data...
Done!


## Configure plotting

In [4]:
%matplotlib ipympl

In [7]:
# Default settings for matplotlib figures
import matplotlib as mpl

# Choose how big to make figures. This will scale the text size.
mpl.rcParams['figure.figsize'] = (5., 4.)

# Choose the size of full-page spectra figures.
spectrum_plot_figsize = (9., 4.)

# Choose how to plot spectra. We use F_nu instead of F_lambda so that
# the features at all wavelengths can be seen. The overall scale is
# arbitrary and normalized to be ~1.
spectrum_plot_scale = a.wave**2 / 5000**2
spectrum_plot_ylabel = 'Normalized flux (erg/$cm^2$/s/Hz)'

# Choose the colormap to use for all of the plots
# from matplotlib.colors import ListedColormap
# crange = np.linspace(0, 1, 256)
# plot_cmap = ListedColormap(plt.cm.plasma(crange)[:210])
plot_cmap = plt.cm.coolwarm

# Set the DPI. This will change how big things appear in Jupyter lab
# mpl.rcParams['figure.dpi'] = 120

# Interpolation plots

## Examples of interpolations

In [6]:
def plot_same_night(idx, save=False, **kwargs):
    night_flux = a.flux[a.target_map == idx]
    phases = a.salt_phases[a.target_map == idx]
    model = a.maximum_result['maximum_flux'][idx]
    model_err = a.maximum_result['maximum_fluxerr'][idx]
    fig1 = plt.figure(**kwargs)
    for flux, phase in zip(night_flux, phases):
        plt.plot(a.wave, flux * spectrum_plot_scale, label='Data (%.2f days)' % phase)
    plt.plot(a.wave, model * spectrum_plot_scale, c='k', ls='--', label='Model (0 days)')
    plt.fill_between(a.wave, (model - model_err) * spectrum_plot_scale,
                     (model + model_err) * spectrum_plot_scale, facecolor='k', alpha=0.3)
    plt.legend()
    plt.title(a.targets[idx])
    plt.ylim(0, None)
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.tight_layout()
    
    if save:
        plt.savefig('./figures/interpolation_model_%s.pdf' % a.targets[idx])
    
    phase_slope = a.maximum_result['phase_slope']
    phase_quadratic = a.maximum_result['phase_quadratic']
    gray_offsets = a.maximum_result['gray_offsets'][a.target_map == idx]
    model_diffs = a.maximum_result['model_diffs'][a.target_map == idx]
    
    fig2 = plt.figure(**kwargs)
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, -2.5*np.log10(flux / model), label='Data (%.2f days)' % phase, c='C%d' % i)
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, model_diff, label='Model (%.2f days)' % phase, c='C%d' % i, ls='--')
    plt.legend(ncol=2, loc=1)
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Difference from maximum light (mag)')
    plt.tight_layout()
    
    if save:
        plt.savefig('./figures/interpolation_difference_%s.pdf' % a.targets[idx])
    
    fig3 = plt.figure(**kwargs)
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(night_flux, phases, gray_offsets, model_diffs)):
        plt.plot(a.wave, -2.5*np.log10(flux / model) - model_diff, label='Residuals (%.2f days)' % phase, c='C%d' % i)
    plt.legend()
    plt.title(a.targets[idx])
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Interpolation residuals (mag)')
    plt.tight_layout()
    
    if save:
        plt.savefig('./figures/interpolation_residuals_%s.pdf' % a.targets[idx])
        
    return fig1, fig2, fig3
    
from ipywidgets import interact
interact(plot_same_night, idx=(0, len(a.targets)-1))

interactive(children=(IntSlider(value=101, description='idx', max=202), Checkbox(value=False, description='sav…

<function __main__.plot_same_night(idx, save=False, **kwargs)>

In [14]:
plot_targets = ['PTF13ayw', 'SN2004gc']
for plot_target in plot_targets:
    target_names = np.array([i.name for i in a.targets])
    plot_idx = np.where(target_names == plot_target)[0][0]

    plot_same_night(plot_idx, save=True)

AttributeError: 'ManifoldTwinsAnalysis' object has no attribute 'interpolation_result'

## Interpolation model

In [ ]:
phase_slope = a.interpolation_result['phase_slope']
phase_quadratic = a.interpolation_result['phase_quadratic']
phase_slope_x1 = a.interpolation_result['phase_slope_x1']
phase_quadratic_x1 = a.interpolation_result['phase_quadratic_x1']

def evaluate_phase_difference(phase, x1=0):
    phase_difference = (
        phase_slope * phase
        + phase_quadratic * phase * phase
        + phase_slope_x1 * x1 * phase
        + phase_quadratic_x1 * x1 * phase * phase
    )
    
    return phase_difference

# Look at change in phase for the same x1
max_phase = a.phase_width
min_phase = -a.phase_width
num_phases = 10
phases = np.linspace(min_phase, max_phase, num_phases)

plt.figure(figsize=spectrum_plot_figsize)
# plt.figure()
norm = plt.Normalize(vmin=min_phase, vmax=max_phase)
cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(phases)

for phase in phases:
    plt.plot(a.wave, evaluate_phase_difference(phase), c=cmap(norm(phase)), zorder=np.abs(phase))
    
plt.colorbar(sm, label='Phase (days)')

# plt.xlim(-5.2, 5.2)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Brightness relative to $t_{max,B}$ (mag)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('./figures/interpolation_phase_difference.pdf')


def plot_x1_difference(phase):
    # Look at change in phase for the same x1
    min_x1 = -2
    max_x1 = 2
    num_x1s = 10
    x1s = np.linspace(min_x1, max_x1, num_x1s)

    plt.figure(figsize=spectrum_plot_figsize)
    norm = plt.Normalize(vmin=min_x1, vmax=max_x1)
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(x1s)

    for x1 in x1s:
        plt.plot(a.wave, evaluate_phase_difference(phase, x1) - evaluate_phase_difference(phase, 0), c=cmap(norm(x1)))

    plt.colorbar(sm, label='SALT2 $x_1$')

    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel('Difference relative to $x_1=0$ (mag)')
    plt.title('Difference in interpolation at %+d days' % phase)
    # plt.gca().invert_yaxis()
    plt.ylim(0.4, -0.4)
    plt.tight_layout()
    plt.savefig('./figures/interpolation_x1_difference_phase_%d.pdf' % phase)
    
for phase in [-5, -3, -1, 1, 3, 5]:
    plot_x1_difference(phase)

## Gray dispersion

In [ ]:
print(a.interpolation_result['gray_dispersion_scale'])

In [ ]:
plt.figure()
plt.scatter(a.salt_phases, a.interpolation_result['gray_offsets'], s=3, label='Individual spectra')
math.plot_binned_mean(a.salt_phases, a.interpolation_result['gray_offsets'], c='C2', lw=2, label='Binned mean')
math.plot_binned_rms(a.salt_phases, a.interpolation_result['gray_offsets'], c='C3', lw=2, label='Binned RMS')
plt.xlabel('SALT2 Phase (days)')
plt.ylabel('Gray offset (mag)')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/gray_offset_vs_phase.pdf')

## Interpolation uncertainty

In [ ]:
try:
    coefs = a.interpolation_result['phase_dispersion_coefficients']
except KeyError:
    coefs = a.stan_data['phase_dispersion_coefficients']
num_phase_coefficients = len(coefs)

def evaluate_phase_dispersion(phase):
    phase_scale = np.abs((num_phase_coefficients / 2) * (phase / a.phase_width))
    full_bins = int(np.floor(phase_scale))
    remainder = phase_scale - full_bins
    
    phase_coefficients = np.zeros(num_phase_coefficients)
    
    for j in range(full_bins + 1):
        if j == full_bins:
            weight = remainder
        else:
            weight = 1
            
        if weight == 0:
            break
            
        if phase > 0:
            phase_bin = num_phase_coefficients // 2 + j
        else:
            phase_bin = num_phase_coefficients // 2 - 1 - j
            
        phase_coefficients[phase_bin] = weight
        
    fractional_dispersion = phase_coefficients.dot(coefs)
    
    # Convert to magnitudes
    mag_dispersion = frac_to_mag(fractional_dispersion)
    
    return mag_dispersion

phases = np.linspace(-a.phase_width, a.phase_width, 1 + num_phase_coefficients)

eval_coefs = np.array([evaluate_phase_dispersion(phase) for phase in phases])

# Uncertainties for different wavelengths
plt.figure()
num_wave = 10
for i in range(num_wave):
    min_wave = a.wave[0]
    max_wave = a.wave[-1]
    wave_range = max_wave - min_wave
    target_wave = min_wave + wave_range * i / (num_wave - 1)
    idx = np.argmin(np.abs(a.wave - target_wave))
    use_wave = a.wave[idx]
    color = plt.cm.rainbow((use_wave - min_wave) / wave_range)
    plt.plot(phases, eval_coefs[:, idx], label='%d $\AA$' % use_wave, c=color)
    
plt.xlim(-5.2, 5.2)
plt.xlabel('Phase (days)')
plt.ylabel('Interpolation uncertainty (mag)')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_phase.pdf')

plt.figure()
for i in range(len(phases)):
    plt.plot(a.wave, eval_coefs[i], label='%.2f days' % phases[i])
plt.legend()
plt.xlabel('Wavelength $(\AA$)')
plt.ylabel('Interpolation uncertainty (mag)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_wavelength.pdf')

## Model accuracy

In [ ]:
max_flux = a.interpolation_result['maximum_flux']
max_fluxerr = a.interpolation_result['maximum_fluxerr']

max_magerr = frac_to_mag(max_fluxerr / max_flux)

rbtl_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])

def plot_uncertainties(show_rbtl=False):
    plt.figure()
    offset = 29
    
    # Make sure that we include the worst offender.
    max_loc = np.argmax(np.sum(max_magerr**2, axis=1))
    start = max_loc % offset
    
    for idx in range(start, len(a.targets), offset):
        plt.plot(a.wave, max_magerr[idx], label=a.targets[idx].name)
    plt.legend(ncol=2)
    
    plt.xlabel('Wavelength ($\AA$)')
    
    if show_rbtl:
        plt.plot(a.wave, rbtl_dispersion, label='SN intrinsic dispersion', c='k', lw=2, ls='--')
        plt.ylabel('Dispersion (mag)')
        path = './figures/interpolation_uncertainty_rbtl.pdf'
    else:
        plt.ylabel('Uncertainty on $f_{max}$ (mag)')
        path = './figures/interpolation_uncertainty_norbtl.pdf'
        
    plt.legend(ncol=2)
    plt.tight_layout()
    plt.savefig(path)
        
plot_uncertainties(False)
plot_uncertainties(True)


plt.figure()
for idx in range(len(a.targets)):
    if idx == 0:
        label = 'Individual uncertainties of $f_{max}$'
    else:
        label = ''
    plt.plot(a.wave, max_magerr[idx], label=label, alpha=0.02, c='C0')
plt.plot(a.wave, rbtl_dispersion, label='Supernova intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.median(max_magerr, axis=0), label='Median uncertainty on $f_{max}$', lw=2, ls='--', c='C0')
plt.plot(a.wave, np.max(max_magerr, axis=0), label='Maximum uncertainty on $f_{max}$', c='C1')
plt.legend()
plt.ylabel('Dispersion (magnitude)')
plt.xlabel('Wavelength ($\AA$)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_median.pdf')

plt.figure()
plt.plot(a.wave, rbtl_dispersion, label='SN intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.min(max_magerr, axis=0), label='Minimum $\sigma_{f,max}$')
for percentile in (25, 50, 75):
    plt.plot(a.wave, np.percentile(max_magerr, percentile, axis=0), label='%dth percentile $\sigma_{f,max}$' % percentile)
plt.plot(a.wave, np.max(max_magerr, axis=0), label='Maximum $\sigma_{f,max}$')
plt.legend(ncol=2)
plt.ylabel('Dispersion (magnitude)')
plt.xlabel('Wavelength ($\AA$)')
plt.tight_layout()
plt.savefig('./figures/interpolation_uncertainty_percentile.pdf')

## Contribution to the total interpolation uncertainty from various sources

In [ ]:
targets = []
diffs = []
phases_1 = []
phases_2 = []
x1s = []
gray_differences = []

gray_offsets = a.interpolation_result['gray_offsets']

center_specs = a.spectra[a.center_mask]
center_gray_offsets = gray_offsets[a.center_mask]
for target_idx in range(len(a.targets)):
    near_max_spec = center_specs[target_idx]
    
    target_mask = (a.target_map == target_idx) & (~a.center_mask)
    target_specs = a.spectra[target_mask]
    target_gray_offsets = gray_offsets[target_mask]
    
    for spec_idx, target_spec in enumerate(target_specs):
        phase_diff = target_spec.phase - near_max_spec.phase
        # if np.abs(phase_diff) < 1:
            # continue

        targets.append(a.targets[target_idx])
        diff = -2.5*np.log10(target_spec.flux / near_max_spec.flux)
        diffs.append(diff)
        phases_1.append(near_max_spec.phase)
        phases_2.append(target_spec.phase)
        x1s.append(a.salt_x1[target_idx])
        
        gray_differences.append(target_gray_offsets[spec_idx] - center_gray_offsets[target_idx])

targets = np.array(targets)
diffs = np.array(diffs)
phases_1 = np.array(phases_1)
phases_2 = np.array(phases_2)
x1s = np.array(x1s)
gray_differences = np.array(gray_differences)

phase_diffs = phases_2 - phases_1

def plot_diffs(diffs, model_subtracted=False):
    sel_mask = np.zeros(len(diffs), dtype=bool)
    sel_mask[4::50] = True
    sel_mask[np.abs(phase_diffs) < 1] = False
    
    print(np.min(x1s[sel_mask]))
    print(np.max(x1s[sel_mask]))
    
    plt.figure(figsize=(8, 5))
    
    for use_idx in np.where(sel_mask)[0]:
        target = targets[use_idx]
        phase_1 = phases_1[use_idx]
        phase_2 = phases_2[use_idx]
        
        if phase_1 > phase_2:
            phase_1, phase_2 = phase_2, phase_1
        
        label = '%s, %.1f to %.1f days' % (target, phase_1, phase_2)
        
        plt.plot(a.wave, diffs[use_idx] / phase_diffs[use_idx], alpha=0.5, label=label)
        
    plt.legend(ncol=2)

    plt.ylim(-0.25, 0.25)
    plt.xlabel('Wavelength ($\AA$)')
    if model_subtracted:
        plt.ylabel('$\Delta m / \Delta t$ (data) - $\Delta m / \Delta t$ (model) (mag/day)')
    else:
        plt.ylabel('$\Delta m / \Delta t$ (data) (mag/day)')
    plt.tight_layout()

plot_diffs(diffs)
plt.savefig('./figures/raw_phase_difference.pdf')

residuals_no_x1 = []
residuals_x1 = []
for diff, phase_1, phase_2, x1 in zip(diffs, phases_1, phases_2, x1s):
    model_no_x1 = evaluate_phase_difference(phase_2, 0) - evaluate_phase_difference(phase_1, 0)
    model_x1 = evaluate_phase_difference(phase_2, x1) - evaluate_phase_difference(phase_1, x1)
    residuals_no_x1.append(diff - model_no_x1)
    residuals_x1.append(diff - model_x1)
    
residuals_no_x1 = np.array(residuals_no_x1)
residuals_x1 = np.array(residuals_x1)

residuals_gray_no_x1 = residuals_no_x1 - gray_differences[:, None]
residuals_gray_x1 = residuals_x1 - gray_differences[:, None]

plot_diffs(residuals_gray_no_x1, True)
plt.savefig('./figures/corr_phase_difference_no_x1.pdf')

plot_diffs(residuals_gray_x1, True)
plt.savefig('./figures/corr_phase_difference_x1.pdf')

def print_interpolation_residuals(min_days, max_days):
    cut = (np.abs(phase_diffs) < max_days) & (np.abs(phase_diffs) > min_days)

    def do_print(label, vals, cut):
        cut_vals = vals[cut]
        print('%20s: std=%.3f, NMAD=%.3f' % (label, math.rms(cut_vals), math.nmad(cut_vals)))

    print("Interpolation of %.1f-%.1f days:" % (min_days, max_days))
    do_print('Raw', diffs, cut)    
    do_print('Phase', residuals_no_x1, cut)    
    do_print('Phase + x1', residuals_x1, cut)    
    do_print('Phase + gray', residuals_gray_no_x1, cut)    
    do_print('Phase + x1 + gray', residuals_gray_x1, cut)    
    print("")
    
print_interpolation_residuals(0., 1.5)
print_interpolation_residuals(1.5, 2.5)
print_interpolation_residuals(2.5, 5.5)
print_interpolation_residuals(5.5, 10.5)

# Example of similarity

In [ ]:
a.maximum_result['gray_offsets']

In [ ]:
phase_slope = a.interpolation_result['phase_slope']

# Reading between the lines plots

## Show spectra before and after

In [ ]:
plt.figure()
plt.plot(a.wave, a.maximum_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
plt.plot(a.wave, a.maximum_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], lw=1.)
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.legend()
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/spectra_at_maximum.pdf')

plt.figure()
plt.plot(a.wave, a.scale_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
plt.plot(a.wave, a.scale_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], alpha=1., lw=1.)
plt.plot(a.wave, a.mean_flux * spectrum_plot_scale, c='k', lw=2, ls='--', label='Mean spectrum')
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/scale_spectra.pdf')

plt.figure()
fractional_dispersion = a.rbtl_result['fractional_dispersion']
plt.plot(a.wave, a.mean_flux * spectrum_plot_scale, c='k', lw=2, ls='--', label='Mean spectrum')
plt.fill_between(a.wave, a.mean_flux * (1 - fractional_dispersion) * spectrum_plot_scale, a.mean_flux * (1 + fractional_dispersion) * spectrum_plot_scale, label='Supernova intrinsic dispersion', alpha=0.5)
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/scale_spectra_model.pdf')

plt.figure()
intrinsic_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])
plt.plot(a.wave, intrinsic_dispersion, c='k', lw=2, label='Supernova intrinsic dispersion')
plt.legend()
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Intrinsic dispersion (mag)')
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/rbtl_intrinsic_dispersion.pdf')

In [ ]:
# Combined version

fig, axes = plt.subplots(3, 1, figsize=(spectrum_plot_figsize[0], spectrum_plot_figsize[1] * 3 - 1.5), sharex=True)

ax = axes[0]
ax.plot(a.wave, a.maximum_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
ax.plot(a.wave, a.maximum_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], lw=1.)
ax.set_ylabel(spectrum_plot_ylabel)
ax.legend()
ax.set_title('Original spectra')
ax.set_ylim(0, None)

ax = axes[1]
ax.plot(a.wave, a.scale_flux[a.interp_mask][0] * spectrum_plot_scale, alpha=1, lw=1., label='Individual spectra')
ax.plot(a.wave, a.scale_flux[a.interp_mask][1:].T * spectrum_plot_scale[:, None], alpha=1., lw=1.)
ax.plot(a.wave, a.mean_flux * spectrum_plot_scale, c='k', lw=2, ls='--', label='Mean spectrum')
ax.legend()
ax.set_ylabel(spectrum_plot_ylabel)
ax.set_title('Dereddened spectra')
ax.set_ylim(0, None)

ax = axes[2]
intrinsic_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])
ax.plot(a.wave, intrinsic_dispersion, c='k', lw=2, label='Supernova intrinsic dispersion')
ax.legend()
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Intrinsic dispersion (mag)')
ax.set_title('Recovered intrinsic dispersion')
ax.set_ylim(0, None)

plt.tight_layout()

plt.savefig('./figures/rbtl_spectra_combined.pdf')


## Magnitudes

In [ ]:
plt.figure()

plt.scatter(a.redshifts[a.interp_mask], a.mags[a.interp_mask], s=15, c='C3', label='Supernovae rejected by cuts')
plt.scatter(a.redshifts[a.good_mag_mask], a.mags[a.good_mag_mask], s=15, c='C0', label='Supernovae passing cuts')

z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')
plt.axvline(0.02, lw=1, ls='--', c='k', label='Redshift cutoff')

plt.xlim(0.001, 0.09)
plt.ylim(-1, 1.5)
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/rbtl_magnitude.pdf')

In [ ]:
plt.figure()

plt.scatter(a.redshifts[a.interp_mask], a.salt_hr[a.interp_mask], s=15, c='C3', label='Supernovae rejected by cuts')
plt.scatter(a.redshifts[a.good_salt_mask], a.salt_hr[a.good_salt_mask], s=15, c='C0', label='Supernovae passing cuts')

z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')
plt.axvline(0.02, lw=1, ls='--', c='k', label='Redshift cutoff')

plt.xlim(0.001, 0.09)
# plt.ylim(-1, 1.5)
plt.ylim(-1, 2.)
plt.xlabel('Redshift')
plt.ylabel('SALT2 measured magnitude')
plt.legend()
plt.tight_layout()
plt.savefig('./figures/salt_magnitude_redshift.pdf')

In [ ]:
plt.figure()

plt.scatter(a.redshifts[a.good_mag_mask], a.mags[a.good_mag_mask], s=15, c='C0', label='Supernovae passing cuts')
plt.xlabel('Redshift')
plt.ylabel('RBTL measured magnitude')


z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')

plt.legend()
plt.tight_layout()
plt.xlim(0.01, 0.09)
plt.ylim(-0.5, 0.5)
plt.savefig('./figures/rbtl_magnitude_cut.pdf')

In [ ]:
print("Raw RBTL mag std:  %.3f mag" % np.std(a.mags[a.good_mag_mask & a.interp_mask]))
print("Raw RBTL mag NMAD: %.3f mag" % math.nmad(a.mags[a.good_mag_mask & a.interp_mask]))

In [ ]:
plt.figure()

plt.scatter(a.redshifts[a.good_mag_mask], a.corr_mags[a.good_mag_mask], s=15, c='C0', label='Supernovae passing cuts')
plt.xlabel('Redshift')
plt.ylabel('RBTL corrected magnitude')


z_range = np.linspace(0.001, 0.09, 100)
pec_vel_disp = 0.00217 / z_range
plt.fill_between(z_range, -pec_vel_disp, pec_vel_disp, alpha=0.2, label='Peculiar velocity dispersion')

plt.legend()
plt.tight_layout()
plt.xlim(0.01, 0.09)
plt.ylim(-0.5, 0.5)
plt.savefig('./figures/rbtl_corr_magnitude_cut.pdf')

# Manifold learning plots

## Plot the parameter space

In [ ]:
# a.scatter(a.embedding[:, 2], vmin=-2, vmax=2, label='Component 3', linewidths=1, edgecolors='k', marker_size=60)
a.scatter(a.embedding[:, 2], vmin=-2, vmax=2, label='Component 3')
plt.savefig('./figures/embedding_components_12.pdf')
a.scatter(a.embedding[:, 1], axis_1=0, axis_2=2, vmin=-3, vmax=3, label='Component 2')
plt.savefig('./figures/embedding_components_13.pdf')
a.scatter(a.embedding[:, 0], axis_1=1, axis_2=2, vmin=-4, vmax=4, label='Component 1')
plt.savefig('./figures/embedding_components_23.pdf')

## Reconstruction uncertainty

In [ ]:
# Note: the total variance isn't defined for Isomap. The variances of the transfomed components
# do map onto the variance of real components though. We provide a very rough estimate of the
# measurement variance for comparison purposes... not sure how much it can be trusted...

num_show = 10

# Do an initial embedding with as many components as possible to get the full variance.
a.generate_embedding(num_components=None)
variances = np.var(a.embedding[a.interp_mask], axis=0)

ref_var = np.sum(variances[:10])

plot_ref = variances[0]

print(variances[:10] / plot_ref)
print(variances[:10] / np.cumsum(variances[:10]))

plt.figure()
plt.scatter(np.arange(num_show), variances[:num_show] / plot_ref, label='Contributed variance of each component')
# plt.scatter(np.arange(num_show), variances[:num_show] / plot_ref, label='Contributed variance of each component')
# plt.axhline(0.1 * ref_var / plot_ref, label='Approximate measurement variance cut', ls='--', c='C3')
# plt.axhline(np.mean(a.interp_power_fraction[a.interp_mask]) * ref_var / plot_ref, label='Approximate mean measurement variance', ls='--', c='C2')
plt.ylim(0, None)
plt.xlabel('Component number')
plt.ylabel('Relative variance explained')
plt.xticks(np.arange(num_show), np.arange(num_show) + 1)
# plt.legend(markerfirst=False)
plt.tight_layout()

plt.savefig('./figures/isomap_component_variance.pdf')

In [ ]:
np.mean(a.interp_power_fraction[a.interp_mask])# * ref_var / plot_ref

## Twin reconstruction

In [ ]:
# Plot where twins and non-twins end up for different number of components.
# We also make a summary plot.
confused_fraction = []

plot_components = np.arange(1, 6)
for n_components in plot_components:
    a.do_embedding(n_components=n_components)
    leakage_matrix = a.plot_twin_distances()
    if n_components == 1:
        title = '1 Component + Color'
    else:
        title = '%d Components + Color' % n_components
    plt.title(title)
    plt.xlabel('Recovered twinness percentile in the embedded space')
    plt.tight_layout()
    plt.savefig('./figures/twins_recovery_%d_components.pdf' % n_components)
    
    confused_fraction.append(leakage_matrix[3, 0] + leakage_matrix[3, 1])

plt.figure()
plt.scatter(np.arange(len(confused_fraction)) + 1, confused_fraction)
plt.xticks(plot_components, plot_components)
plt.ylim(0, 0.1)
plt.xlabel('Number of components (in addition to color)')
plt.ylabel('Fraction of non-twins confused as twins')
plt.tight_layout()
plt.savefig('./figures/twins_confusion.pdf')

## Plot slices through the manifold

In [27]:
a.generate_embedding()

def plot_slice(scan_component, closest_count=10, max_dist=1., loc=np.zeros(a.embedding.shape[1] - 1)):
    loc = np.asarray(loc)
    mask = a.interp_mask

    use_embedding = a.embedding[mask]
    use_flux = a.scale_flux[mask]

    other_embedding = np.delete(use_embedding, scan_component, axis=1)
    dists = np.sqrt(np.sum((other_embedding - loc)**2, axis=1))

    dist_limit = np.min([np.sort(dists)[closest_count], max_dist])
    scan_cut = dists < dist_limit
    
    scan_embedding = use_embedding[scan_cut, scan_component]
    
    sort_embedding = np.sort(scan_embedding)
    min_comp = sort_embedding[0]
    max_comp = sort_embedding[-1]
    cmap = plot_cmap

    plt.figure(figsize=spectrum_plot_figsize)
    for spec, val in zip(use_flux[scan_cut], scan_embedding):
        plt.plot(a.wave, spec * spectrum_plot_scale, c=cmap((val - min_comp) / (max_comp - min_comp)))

    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_comp, vmax=max_comp))
    sm._A = []
    plt.colorbar(sm, label='Value of Component %d' % (scan_component + 1))
    plt.title('Component %d' % (scan_component + 1))
    plt.tight_layout()
    
    plt.savefig('./figures/component_%d_effect.pdf' % (scan_component + 1))

Generating the manifold learning embedding...


In [ ]:
plot_slice(0, loc=[-0.5, -0.5])
plot_slice(1)
plot_slice(2, loc=[0, 1])

In [37]:
a.plot_gp()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot steps through component values - combined

In [34]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(3, 1, figsize=(spectrum_plot_figsize[0], spectrum_plot_figsize[1] * 3 - 1.5), sharex=True)

num_steps = 10

for component in range(3):
    mask = a.uncertainty_mask

    use_embedding = a.embedding[mask, component]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    ax = axes[component]
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []
    
    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        # Make the extreme values of components get plotted on top if everything overlaps.
        zorder = np.abs(mean_val)
        
        ax.plot(a.wave, step_flux * spectrum_plot_scale, c=sm.to_rgba(mean_val), zorder=zorder)
        
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2.5%', pad=0.1)
    fig.colorbar(sm, cax=cax, orientation='vertical', label='Component Value')
    # fig.colorbar(sm, cax=cax, orientation='vertical', label='Value of Component %d' % (component + 1))
    # ax.colorbar(sm, label='Value of Component %d' % (component + 1))
    ax.set_title('Component %d' % (component + 1))
    
    if component == 2:
        ax.set_xlabel('Wavelength ($\AA$)')
    
    ax.set_ylabel(spectrum_plot_ylabel)
    ax.set_ylim(0, None)
    
plt.tight_layout()

plt.savefig('./figures/component_steps_combined.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
# Version for talks
from mpl_toolkits.axes_grid1 import make_axes_locatable

# fig, axes = plt.subplots(3, 1, figsize=(spectrum_plot_figsize[0], spectrum_plot_figsize[1] * 3 - 1.5), sharex=True)
fig, axes = plt.subplots(3, 1, figsize=(9, 7), sharex=True)

num_steps = 10

for component in range(3):
    mask = a.uncertainty_mask

    use_embedding = a.embedding[mask, component]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    ax = axes[component]
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []
    
    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        # Make the extreme values of components get plotted on top if everything overlaps.
        zorder = np.abs(mean_val)
        
        ax.plot(a.wave, step_flux * spectrum_plot_scale, c=sm.to_rgba(mean_val), zorder=zorder)
        
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2.5%', pad=0.1)
    fig.colorbar(sm, cax=cax, orientation='vertical', label='Component Value')
    # fig.colorbar(sm, cax=cax, orientation='vertical', label='Value of Component %d' % (component + 1))
    # ax.colorbar(sm, label='Value of Component %d' % (component + 1))
    # ax.set_title('Component %d' % (component + 1))
    
    if component == 2:
        ax.set_xlabel('Wavelength ($\AA$)')
    
    # ax.set_ylabel(spectrum_plot_ylabel)
    ax.set_ylabel('Flux')
    ax.set_ylim(0, None)
    
plt.tight_layout()

plt.savefig('./figures/component_steps_combined_talk.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot steps through component values - single

In [ ]:
def plot_steps(component, num_steps=10, xlim=None, figsize=spectrum_plot_figsize, colorbar=True):
    mask = a.uncertainty_mask

    use_embedding = a.embedding[mask, component]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    plt.figure(figsize=figsize)
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []

    if xlim is not None:
        wave_mask = (a.wave > xlim[0] - 50) & (a.wave < xlim[1] + 50)
    else:
        wave_mask = np.ones(len(a.wave), dtype=bool)
    
    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        # if step == 0:
            # label = 'Median spectra in each component bin'
        # else:
            # label = ''
            
        # Make the extreme values of components get plotted on top if everything overlaps.
        zorder = np.abs(mean_val)
        
        # plt.plot(a.wave[wave_mask], step_flux[wave_mask], c=sm.to_rgba(mean_val), label=label)
        plt.plot(a.wave[wave_mask], step_flux[wave_mask] * spectrum_plot_scale[wave_mask], c=sm.to_rgba(mean_val), zorder=zorder)
        
    if xlim is not None:
        plt.xlim(*xlim)
        
    if colorbar:
        plt.colorbar(sm, label='Value of Component %d' % (component + 1))
        plt.title('Component %d' % (component + 1))
    
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.ylim(0, None)
    
    # plt.legend()
    
    plt.tight_layout()
    
    if xlim is None:
        plt.savefig('./figures/component_%d_steps.pdf' % (component + 1))
    else:
        plt.savefig('./figures/component_%d_steps_zoom_%d_%d.pdf' % (component + 1, xlim[0], xlim[1]))

In [ ]:
for component in range(3):
    plot_steps(component)
    # plot_steps(component, xlim=(3300, 4500))
    # plot_steps(component, xlim=(4900, 6700))
    # plot_steps(component, xlim=(7200, 8600))

## Comparison to original twins

In [ ]:
hannah_list = np.genfromtxt('./data/fakhouri_atmax_list.txt', dtype='str')
twins_mask = np.array([i.name in hannah_list for i in a.targets])

In [ ]:
a.plot_twin_pairings()
plt.xlim(0, 100)
plt.ylim(0, None)
plt.tight_layout()
plt.savefig('./figures/twin_dispersion.pdf')

## Twins that are poor brightness matches

In [ ]:
from scipy.spatial.distance import pdist, squareform

mask = a.good_mag_mask

raw_spec_dists = pdist(a.iso_diffs[mask])
spec_dists = squareform(raw_spec_dists)
mag_diffs = squareform(pdist(a.mags[mask][:, None]))

In [ ]:
dist_mask = spec_dists < np.percentile(raw_spec_dists, 20)
plt.figure()
plt.hist(mag_diffs[dist_mask])

In [ ]:
idx1, idx2 = np.where(dist_mask & (mag_diffs > 0.35))
idx_mask = idx1 < idx2
idx1 = idx1[idx_mask]
idx2 = idx2[idx_mask]

def print_pairs(vals):
    for val_name, val in vals.items():
        print("%13s_1" % val_name, "%13s_2" % val_name, end='')
    print('')
        
    for i, j in zip(idx1, idx2):
        for val_name, val in vals.items():
            try:
                print('%15.3f' % val[mask][i], '%15.3f' % val[mask][j], end='')
            except TypeError:
                print('%15s' % val[mask][i], '%15s' % val[mask][j], end='')
        print('')

print_pairs({
    'target': a.targets,
    'redshift': a.redshifts,
    'color': a.colors,
    'mag': a.mags,
    'salt_mag': a.salt_hr,
    'embedding[0]': a.embedding[:, 0],
})

## Comparision to Branch classifications

In [ ]:
a.do_blondin_plot()
plt.savefig('./figures/branch_classification.pdf')

In [ ]:
a.do_component_blondin_plot(axis_1=0, axis_2=2)
plt.gca().get_legend().remove()

In [ ]:
a.do_component_blondin_plot(axis_1=1, axis_2=2)
plt.gca().get_legend().remove()

In [ ]:
a.do_component_blondin_plot()
plt.gca().get_legend().remove()
plt.savefig('./figures/branch_labels_isomap.pdf')

## Variation of Core Normal SNe Ia

In [ ]:
indicators = a.get_indicators() 

s1 = indicators["EWSiII6355"]
s2 = indicators["EWSiII5972"]

component = 0

core_normal_cut = (s2 < 30) & (s1 > 70) & (s1 < 100) & ~np.isnan(a.embedding[:, component])

cut_flux = a.scale_flux[core_normal_cut]
cut_coord = a.embedding[core_normal_cut][:, component]

sort_flux = cut_flux[np.argsort(cut_coord)]
sort_coord = cut_coord[np.argsort(cut_coord)]

num_bins = 5

cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=np.percentile(cut_coord, 100 / num_bins / 2), vmax=np.percentile(cut_coord, 100 * (1 - 1. / num_bins / 2))))
sm._A = []

def plot_spec(bin_idx):
    min_idx = int(len(sort_coord) / num_bins * bin_idx)
    max_idx = int(len(sort_coord) / num_bins * (bin_idx + 1))
    bin_flux = sort_flux[min_idx:max_idx]
    
    f = np.median(bin_flux, axis=0)
    
    mean_val = np.mean(sort_coord[min_idx:max_idx])
    
    plt.plot(a.wave, f * spectrum_plot_scale, c=sm.to_rgba(mean_val), zorder=np.abs(mean_val))
    
plt.figure(figsize=spectrum_plot_figsize)
for i in range(num_bins):
    plot_spec(i)
plt.colorbar(sm, label='Value of Component %d' % (component + 1))
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
# plt.legend()
plt.tight_layout()
plt.savefig('./figures/core_normal_comparison.pdf')

## Recovering other indicators of intrinsic diversity

In [ ]:
# Set up an array to hold everything
all_indicators = []

In [ ]:
# Spectral features
indicators = a.spectral_indicators

name_map = {
    'EWCaIIHK': 'pEW Ca II HK',
    'EWSiII4000': 'pEW Si II 4000',#$\AA$',
    'EWSiII5972': 'pEW Si II 5972',#$\AA$',
    'EWSiII6355': 'pEW Si II 6355',#$\AA$',
    'vCaIIHK': 'Velocity Ca II HK',
    'vSiII6355': 'Velocity Si II 6355',#$\AA$',
    'lamCaIIHK': 'Lambda Ca II HK',
    'lamSiII6355': 'Lambda Si II 6355',#$\AA$',
}

for key in indicators.keys():
    values = indicators[key]

    if key[:3] == 'lam':
        continue
        
    if key[:1] == 'v':
        # Use km/s and make everything positive.
        values = values / -1000

    all_indicators.append((name_map[key], values, a.uncertainty_mask))

In [ ]:
# SALT2 X1
all_indicators.append(('SALT2 $x_1$', a.salt_x1, a.salt_mask))

In [ ]:
import pickle

# Sugar parameters
pickle_data =  open('./data/sugar_parameters.pkl').read().replace('\r\n', '\n').encode('latin1')
sugar_data = pickle.loads(pickle_data, encoding='latin1')

sugar_rows = []
for target in a.targets:
    try:
        row = sugar_data[target.name.encode('latin1')]
        sugar_rows.append([row['q1'], row['q2'], row['q3']])
    except KeyError:
        sugar_rows.append([np.nan] * 3)

sugar_embedding = np.array(sugar_rows, dtype=float)
sugar_mask = ~np.isnan(sugar_embedding[:, 0])

for component in range(3):
    all_indicators.append(('SUGAR Component %d\n(Leget et al. 2019)' % (component + 1), sugar_embedding[:, component], sugar_mask))

In [ ]:
# Nordin colors
nordin_bands = {
    'uNi': (3300., 3510.),
    'uTi': (3510., 3660.),
    'uSi': (3660., 3760.),
    'uCa': (3750., 3860.),
}

for label, (wave_min, wave_max) in nordin_bands.items():
    cut = (a.wave > wave_min) & (a.wave < wave_max)
    
    values = -2.5*np.log10(np.sum(a.scale_flux[:, cut], axis=1) / np.sum(a.mean_flux[cut]))
    
    all_indicators.append(('U-band variation—%s\n(Nordin et al. 2018)' % label, values, a.uncertainty_mask))

In [ ]:
# SNEMO parameters
import pandas as pd
snemo_data = pd.read_csv('./data/snemo_salt_coefficients_snf.csv').set_index('SN')

nan_row = snemo_data.iloc[0].copy()
nan_row[:] = np.nan

snemo_rows = []
for target in a.targets:
    try:
        row = snemo_data.loc[target.name]
        snemo_rows.append(row)
    except KeyError:
        snemo_rows.append(nan_row)

snemo_embedding = pd.DataFrame(snemo_rows)
snemo_mask = ~np.isnan(snemo_embedding['salt_c'])
snemo_labels = snemo_data.columns

# SNEMO 2
all_indicators.append(('SNEMO2 Component 1\n(Saunders et al. 2018)', snemo_embedding['snemo2_c1'], snemo_mask))

# SNEMO 7
for component in range(6):
    all_indicators.append(('SNEMO7 Component %d' % (component + 1), snemo_embedding[f'snemo7_c{component+1}'], snemo_mask))
    
# SNEMO 15
# for component in range(14):
    # all_indicators.append(('SNEMO15 Component %d' % (component + 1), snemo_embedding[f'snemo15_c{component+1}'], snemo_mask))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def find_rotation(values, mask):
    # Find the best predictor of an indicator
    def to_min(x):
        diff = values - a.embedding.dot(x[1:]) - x[0]
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0, 0, 0, 0])

    rotation = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
    best_guess = a.embedding.dot(res.x[1:]) + res.x[0]
    
    return rotation, best_guess

def find_rotation_quadratic(values, mask):
    # Find the best predictor of an indicator
    def evaluate(x):
        e = a.embedding.T
        
        model = (
            x[0]
            + e[0] * x[1]
            + e[1] * x[2]
            + e[2] * x[3]
            + e[0] * e[0] * x[4]
            + e[1] * e[1] * x[5]
            + e[2] * e[2] * x[6]
            + e[0] * e[1] * x[7]
            + e[0] * e[2] * x[8]
            + e[1] * e[2] * x[9]
        )
        
        return model
        
    def to_min(x):
        model = evaluate(x)
        diff = values - model
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0] * 10)

    best_guess = evaluate(res.x)
    
    return res.x, best_guess

names = []
data = []

for name, values, mask in all_indicators[::-1]:
    mask = mask & a.uncertainty_mask
    
    rotation, best_guess = find_rotation(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    
    params_quadratic, best_guess_quadratic = find_rotation_quadratic(values, mask)
    corrcoef_quadratic = np.corrcoef(best_guess_quadratic[mask], values[mask])[0, 1]
    
    names.append(name)
    data.append(np.hstack([rotation, corrcoef, corrcoef_quadratic]))

    # print(f'{name} & {rotation[0]:.2f} & {rotation[1]:.2f} & {rotation[2]:.2f} & {corrcoef:.2f}')
    
data = np.array(data)

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(5.0, 8))
plt.subplots_adjust(wspace=0., hspace=0.)

cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=-1, vmax=1))
sm._A = []

def do_plot(ax, ax_data, xlabels, ylabels=None):
    im = ax.imshow(ax_data, interpolation='nearest', cmap=cmap, vmin=-1, vmax=1)
    ax.set(
        xticks=np.arange(ax_data.shape[1]),
        yticks=np.arange(ax_data.shape[0]),
        xticklabels=xlabels,
    )

    if ylabels is not None:
        ax.set_yticklabels(ylabels)
    else:
        ax.tick_params(axis='y', which='both', left=False, right=False)

    ax.set_xlim(-0.5, ax_data.shape[1] - 0.5)
    ax.set_ylim(-0.5, ax_data.shape[0] - 0.5)

    # Ticks on top
    ax.get_xaxis().set_ticks_position('top')
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=70, ha="left", va='center',
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = 0.65
    for i in range(ax_data.shape[0]):
        for j in range(ax_data.shape[1]):
            ax.text(j, i, format(ax_data[i, j], fmt),
                    ha="center", va="center",
                    color="white" if np.abs(ax_data[i, j]) > thresh else "black")
            
do_plot(axes[0], data[:, :3], ['Component 1 Rotation', 'Component 2 Rotation', 'Component 3 Rotation'], names)
do_plot(axes[1], data[:, 3:], ['Linear Transformation\nPearson Correlation', 'Quadratic Transformation\nPearson Correlation'])

divider = make_axes_locatable(axes[-1])
cax = divider.append_axes('right', size='30%', pad=0.23)
fig.colorbar(sm, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig('./figures/indicators_recovery.pdf')

In [ ]:
from ipywidgets import interact

def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & a.uncertainty_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
        
    min_val = np.min([values[mask], best_guess[mask]])
    max_val = np.max([values[mask], best_guess[mask]])
    val_range = max_val - min_val
    
    plt.figure(figsize=(4, 3.8))
    plt.scatter(best_guess[mask], values[mask])
    
    plt.text(min_val + 0.*val_range, max_val - 0.02*val_range, "$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    print("$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    
    plt.plot([min_val - val_range, min_val + val_range], [min_val - val_range, min_val + val_range], c='k', ls='--')

    plt.xlabel('Transformation of the Isomap latent space')
    
    plt.ylabel(name)
    # plt.ylabel("Original measurement")
    # plt.title(name)
    
    plt.xlim(min_val-0.05*val_range, max_val+0.05*val_range)
    plt.ylim(min_val-0.05*val_range, max_val+0.05*val_range)
    plt.tight_layout()
    
    
interact(plot_component, idx=(0, len(data) - 1), quadratic=True)

In [ ]:
def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & a.uncertainty_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
        
    min_val = np.min([values[mask], best_guess[mask]])
    max_val = np.max([values[mask], best_guess[mask]])
    val_range = max_val - min_val
    
    plt.figure(figsize=(2.5, 2.7))
    plt.scatter(best_guess[mask], values[mask], s=10)
    
    plt.text(min_val + 0.*val_range, max_val - 0.04*val_range, "$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    print("$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    
    plt.plot([min_val - val_range, min_val + val_range], [min_val - val_range, min_val + val_range], c='k', ls='--')

    # plt.xlabel('Transformation of the Isomap latent space')
    plt.xticks([])
    plt.yticks([])
    
    # plt.ylabel(name)
    # plt.ylabel("Original measurement")
    plt.title(name.split('\n')[0])
    
    plt.xlim(min_val-0.05*val_range, max_val+0.05*val_range)
    plt.ylim(min_val-0.05*val_range, max_val+0.05*val_range)
    plt.tight_layout()

interact(plot_component, idx=(0, len(data) - 1), quadratic=True)

In [ ]:
for i in range(len(data)):
    plot_component(i, quadratic=True)
    plt.savefig('./figures/rotation_%d.png' % i)

In [ ]:
plt.ylabel('SALT2 $x_1$')

In [ ]:
plt.figure()
plt.text(-0.3, 0.6, 'hi')
plt.show()

In [ ]:
plot_component(6, quadratic=True)

In [ ]:
from ipywidgets import interact

def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & a.uncertainty_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
    
    plt.figure()
    plt.scatter(best_guess[mask], values[mask])
    plt.title(name)
    plt.xlabel('Transformation of the Isomap latent space')
    plt.ylabel(name)
    
interact(plot_component, idx=(0, len(data) - 1))

### Plots for AAS

In [ ]:
a.scatter(a.salt_x1, mask=a.salt_mask, label='SALT2 $x_1$')

### TODO: Figure out what happened with SNBOSS38. Somehow the SALT2 fit is way off, but it doesn't fail the SALT2 fit cuts. Reject in manually for now.

In [ ]:
plt.figure()
plt.scatter(a.embedding[a.salt_mask, 1], a.salt_x1[a.salt_mask])

In [ ]:
mask = (a.embedding[:, 0] > 2) & (a.embedding[:, 1] > 1) & (a.embedding[:, 1] < 3)

In [ ]:
for i in np.where(mask)[0]:
    print(a.targets[i], a.embedding[i])

In [ ]:
a.salt_mask[(a.salt_x1 > 1) & (a.embedding[:, 1] > 1)] = False

In [ ]:
a.salt_hr[mask]

In [ ]:
res = tt[2].fit_salt(plot=True)

### Figure out what correlation coefficient we would expect

In [ ]:
all_corrcoef_linear = []
all_corrcoef_quadratic = []

for i in tqdm.tqdm(range(200)):
    mask = a.interp_mask
    values = np.random.normal(size=len(values))
    
    rotation, best_guess = find_rotation(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    all_corrcoef_linear.append(corrcoef)
    
    rotation, best_guess = find_rotation_quadratic(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    all_corrcoef_quadratic.append(corrcoef)

with open('./latex/correlation_simulation.tex', 'w') as f:
    latex_command(f, 'correlationmeanlinear', '%.2f', np.mean(all_corrcoef_linear))
    latex_command(f, 'correlationstdlinear', '%.2f', np.std(all_corrcoef_linear))
    latex_command(f, 'correlationmeanquadratic', '%.2f', np.mean(all_corrcoef_quadratic))
    latex_command(f, 'correlationstdquadratic', '%.2f', np.std(all_corrcoef_quadratic))

### Plot rotated spectra

In [ ]:
from ipywidgets import interact

def plot_steps(idx, quadratic=False):
    num_steps = 10
    name, values, mask = all_indicators[idx]

    mask = mask & a.interp_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
    
    use_embedding = best_guess[mask]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    plt.figure(figsize=spectrum_plot_figsize)
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []

    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        plt.plot(a.wave, step_flux * spectrum_plot_scale, c=sm.to_rgba(mean_val))
        
    plt.colorbar(sm, label='Rotated Isomap vSi II 6355$\AA$\nEstimate ($10^3$ km/s)')
    
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.ylim(0, None)
    plt.title(name)
    
    
    plt.tight_layout()
    
interact(plot_steps, idx=(0, len(data) - 1))

### Linear combinations of SUGAR components

In [ ]:
all_indicators.append(('Isomap Component 1', a.embedding[:, 0], a.interp_mask))
all_indicators.append(('Isomap Component 2', a.embedding[:, 1], a.interp_mask))
all_indicators.append(('Isomap Component 3', a.embedding[:, 2], a.interp_mask))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def find_rotation(values, mask):
    # Find the best predictor of an indicator
    def to_min(x):
        diff = values - sugar_embedding.dot(x[1:]) - x[0]
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0, 0, 0, 0])

    rotation = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
    best_guess = sugar_embedding.dot(res.x[1:]) + res.x[0]
    
    return rotation, best_guess

def find_rotation_quadratic(values, mask):
    # Find the best predictor of an indicator
    def evaluate(x):
        e = sugar_embedding.T
        
        model = (
            x[0]
            + e[0] * x[1]
            + e[1] * x[2]
            + e[2] * x[3]
            + e[0] * e[0] * x[4]
            + e[1] * e[1] * x[5]
            + e[2] * e[2] * x[6]
            + e[0] * e[1] * x[7]
            + e[0] * e[2] * x[8]
            + e[1] * e[2] * x[9]
        )
        
        return model
        
    def to_min(x):
        model = evaluate(x)
        diff = values - model
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0] * 10)

    best_guess = evaluate(res.x)
    
    return res.x, best_guess

names = []
data = []

for name, values, mask in all_indicators[::-1]:
    mask = mask & sugar_mask
    
    rotation, best_guess = find_rotation(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    
    params_quadratic, best_guess_quadratic = find_rotation_quadratic(values, mask)
    corrcoef_quadratic = np.corrcoef(best_guess_quadratic[mask], values[mask])[0, 1]
    
    names.append(name)
    data.append(np.hstack([rotation, corrcoef, corrcoef_quadratic]))

    # print(f'{name} & {rotation[0]:.2f} & {rotation[1]:.2f} & {rotation[2]:.2f} & {corrcoef:.2f}')
    
data = np.array(data)

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(5.0, 8))
plt.subplots_adjust(wspace=0., hspace=0.)

cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=-1, vmax=1))
sm._A = []

def do_plot(ax, ax_data, xlabels, ylabels=None):
    im = ax.imshow(ax_data, interpolation='nearest', cmap=cmap, vmin=-1, vmax=1)
    ax.set(
        xticks=np.arange(ax_data.shape[1]),
        yticks=np.arange(ax_data.shape[0]),
        xticklabels=xlabels,
    )

    if ylabels is not None:
        ax.set_yticklabels(ylabels)
    else:
        ax.tick_params(axis='y', which='both', left=False, right=False)

    ax.set_xlim(-0.5, ax_data.shape[1] - 0.5)
    ax.set_ylim(-0.5, ax_data.shape[0] - 0.5)

    # Ticks on top
    ax.get_xaxis().set_ticks_position('top')
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=70, ha="left", va='center',
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = 0.65
    for i in range(ax_data.shape[0]):
        for j in range(ax_data.shape[1]):
            ax.text(j, i, format(ax_data[i, j], fmt),
                    ha="center", va="center",
                    color="white" if np.abs(ax_data[i, j]) > thresh else "black")
            
do_plot(axes[0], data[:, :3], ['Component 1 Rotation', 'Component 2 Rotation', 'Component 3 Rotation'], names)
do_plot(axes[1], data[:, 3:5], ['Linear Rotation\nPearson Correlation', 'Quadratic Rotation\nPearson Correlation'])

divider = make_axes_locatable(axes[-1])
cax = divider.append_axes('right', size='30%', pad=0.23)
fig.colorbar(sm, cax=cax, orientation='vertical')

plt.tight_layout()
# plt.savefig('./figures/indicators_recovery.pdf')

In [ ]:
from ipywidgets import interact

def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & sugar_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
    
    plt.figure()
    plt.scatter(best_guess[mask], values[mask], c=a.colors[mask], vmin=-0.2, vmax=0.2)
    plt.title(name)
    
interact(plot_component, idx=(0, len(data) - 1))

## Color comparison

In [15]:
a.scatter(a.rbtl_colors, vmin=-0.3, vmax=0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
plt.scatter(-0.54*sugar_embedding[:, 0] + 0.84 * sugar_embedding[:, 2], a.salt_x1)

In [ ]:
plt.figure()
plt.scatter(a.embedding[:, 1], a.colors, c=a.embedding[:, 0])

In [42]:
plt.figure()
m = (a.redshifts > 0.02) & (a.redshift_errs < 0.004) & (a.uncertainty_mask)
plt.scatter(a.rbtl_colors[m] - np.median(a.rbtl_colors[m]), a.corr_mags[m])

np.std(a.corr_mags[m & (a.rbtl_colors - np.median(a.rbtl_colors) > 0.5) & a.train_mask])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.29906577117714916

In [57]:
plt.figure()
plt.plot(a.wave, np.sqrt(np.mean(utils.frac_to_mag((a.maximum_fluxerr / a.maximum_flux)[a.uncertainty_mask])**2, axis=0)))

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
plt.figure()
plt.plot(a.wave, 1/a.rbtl_result['fractional_dispersion']**2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
plt.figure()
plt.scatter(a.embedding[a.uncertainty_mask, 0], a.embedding[a.uncertainty_mask, 1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
a.spectra[(a.trans)]

In [26]:
plt.scatter(0.548, -0.830)

In [37]:
np.sum((a.rbtl_colors - np.median(a.rbtl_colors)) > 0.5)

21

In [ ]:
np.std(a.)

In [35]:
a.targets[dists < 0.1]

array([Target(name="LSQ12gxj"), Target(name="PTF10wnm")], dtype=object)

In [34]:
plt.figure()
plt.plot(a.wave, a.scale_flux[dists < 0.1].T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
dists = np.sum((a.embedding - a.embedding[m & (a.corr_mags > 0.5)])**2, axis=1)

/home/kyle/packages/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


## 91T/91bg labels

In [46]:
outlier_labels = {
    # Scalzo++ 2014
    "SNF20070528-003": "91T-like",
    "SNF20070803-005": "91T-like",
    "SNF20070825-001": "91T-like",
    "SNF20070912-000": "91T-like",
    "SNF20080522-000": "91T-like",
    "SNF20080723-012": "91T-like",
    
    # Lin++ 2020
    "SNF20080805-007": "91T-like",
    "LSQ12cyz": "91T-like",
    "LSQ12fhe": "91T-like",
    "PTF11bju": "91T-like",
    "PTF11mkx": "91T-like",
    "LSQ12cfx": "91bg-like",
    
    # Discussion between Kyle and Greg:
    "SNNGC2691": "91T-like",
    
    # Maguire++ 2011
    "PTF10ops": "91bg-like",
    
    # Lin++ 2020
    "PTF11bkf": "91bg-like",
    "PTF11kjn": "91bg-like",
    "PTF11okh": "91bg-like",
    "PTF11pra": "91bg-like",
    "PTF12dwm": "91bg-like",
    "SN2005bl": "91bg-like",
    "SN2005dh": "91bg-like",
    "SN2005dm": "91bg-like",
    "SN2007ba": "91bg-like",
    "SN2009hs": "91bg-like",
    "SNNGC6430": "91bg-like",
    "SN2005cc": "02cx-like",
    
    # Foley++ 2013
    "SN2011ay": "02cx-like",
    "LSQ12fhs": "02cx-like",
}

In [47]:
a.targets[a.embedding[:, 0] > 6]

array([Target(name="LSQ12fhs"), Target(name="SN2011ay"),
       Target(name="SNF20070528-003")], dtype=object)

In [48]:
plt.figure()

colors = []
labels = []
for idx, sn in enumerate(a.targets):
    if not a.uncertainty_mask[idx]:
        continue

    label = outlier_labels.get(sn.name, 'Other')
    if label == 'Other':
        c = 'silver'
    elif label == '91bg-like':
        c = 'C0'
    elif label == '91T-like':
        c = 'C2'
    elif label == '02cx-like':
        c = 'C3'
        
    if c in colors:
        plot_label = ''
    else:
        plot_label = label

    colors.append(c)
    
    plt.scatter(a.embedding[idx, 0], a.embedding[idx, 1], label=plot_label, facecolors=c)
plt.legend()
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
a.targets[np.array([i.name == 'SNF20070825-001' for i in a.targets])]

array([], dtype=object)

In [32]:
a.plot_gp()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
mask_91t = np.array([outlier_labels.get(i.name, 'aa') == '91T-like' for i in a.targets])

In [31]:
np.sum(mask_91t & a.uncertainty_mask)

7

In [30]:
np.sum(mask_91t & a.redshift_color_mask & a.uncertainty_mask & a.train_mask)

3

In [ ]:
a.scatter(a.corr_mags, mask=a.good_mag_mask)

In [ ]:
for idx in np.where((a.embedding[:, 0] > 4) & (a.embedding[:, 1] > -1.5))[0]:
    target = a.targets[idx]
    print(a.embedding[idx], target, outlier_labels.get(target.name, 'Other'), a.corr_mags[idx])

# SALT2 comparison

In [ ]:
# Load SALT2 Hubble residuals
a.calculate_salt_hubble_residuals()

## SALT2 colors

In [ ]:
plt.figure()
plt.scatter(a.salt_color, a.colors, s=5)
plt.xlabel('SALT2 Color ($c$)')
plt.ylabel('RBTL Color ($A_V$)')
plt.tight_layout()

plt.savefig('./figures/salt2_color_comparison.pdf')

## SALT2 X1

In [ ]:
a.scatter(a.salt_x1, a.interp_mask, label='SALT $x_1$')
plt.savefig('./figures/salt2_x1_components_full.pdf')

In [ ]:
a.scatter(a.salt_x1, a.interp_mask & a.good_salt_mask, axis_1=0, axis_2=1, label='SALT $x_1$', vmin=-2., vmax=1.5)
plt.savefig('./figures/salt2_x1_components.pdf')

In [ ]:
a.plot_gp(kind='salt_raw', vmin=-0.5, vmax=0.5)

In [ ]:
# Find the best predictor of x1
def to_min(x):
    diff = a.salt_x1 - a.embedding.dot(x)
    return np.nanstd(diff[a.salt_mask])

res = minimize(to_min, [0, 0, 0])

norm_x = res.x / np.sqrt(np.sum(res.x**2))
print(norm_x)

plt.figure()
# plt.scatter(a.embedding.dot(res.x), a.salt_x1, c=a.salt_mask)
plt.scatter(a.embedding.dot(res.x)[~a.salt_mask], a.salt_x1[~a.salt_mask], c='C3', s=30, label='"Bad" SALT2 fits', alpha=0.8)
plt.scatter(a.embedding.dot(res.x)[a.salt_mask], a.salt_x1[a.salt_mask], c='C0', s=30, label='"Good" SALT2 fits', alpha=0.8)
plt.plot([-3, 3], [-3, 3], ls='--', c='k', label='One-to-one line')
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.xlabel('Rotated Isomap components')
plt.ylabel('SALT2 $x_1$')
plt.legend()
plt.savefig('./figures/rotated_isomap_salt_x1.pdf')


print(np.corrcoef(a.embedding.dot(res.x)[a.interp_mask & a.salt_mask], a.salt_x1[a.interp_mask & a.salt_mask]))

## SALT2 outliers (Type Iax)

In [ ]:
a.scatter(a.salt_x1)

In [ ]:
iax_mask = (a.embedding[:, 0] > 4.) & (a.embedding[:, 1] < -3)
a.targets[iax_mask]

In [ ]:
# Outlier spectra
mask = (a.embedding[:, 0] > 4) & (a.embedding[:, 1] < -2)
print(a.targets[mask])
print(a.colors[mask])
print(a.redshifts[mask])
print(a.mags[mask])

# Ref spectrum
ref_target = 'SNF20070803-005'
for idx2, target in enumerate(a.targets):
    if target.name == ref_target:
        break
        
# idx2 = np.where((a.embedding[:, 0] > 4.8) & (a.embedding[:, 1] < 2))[0][0]

plt.figure()
for i in np.where(mask)[0]:
    plt.plot(a.wave, a.scale_flux[i] * spectrum_plot_scale, label=a.targets[i].name)
    
plt.plot(a.wave, a.scale_flux[idx2] * spectrum_plot_scale, c='k', ls='--', label=a.targets[idx2].name)
print(a.embedding[idx2])

plt.legend()

plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.tight_layout()

plt.savefig('./figures/type_iax_comparison.pdf')

## Other outliers

In [ ]:
a.targets[(a.embedding[:, 1] > 4.)]

In [ ]:
# Outlier spectra
mask = (a.embedding[:, 0] < -4)
print(a.targets[mask])
print(a.colors[mask])
print(a.redshifts[mask])
print(a.mags[mask])

# Ref spectrum
# ref_target = 'SNF20070803-005'
# for idx2, target in enumerate(a.targets):
    # if target.name == ref_target:
        # break
        
# idx2 = np.where((a.embedding[:, 0] > 4.8) & (a.embedding[:, 1] < 2))[0][0]

plt.figure()
for i in np.where(mask)[0]:
    plt.plot(a.wave, a.scale_flux[i] * spectrum_plot_scale, label=a.targets[i].name)
    
# plt.plot(a.wave, a.scale_flux[idx2] * spectrum_plot_scale, c='k', ls='--', label=a.targets[idx2].name)
print(a.embedding[idx2])

plt.legend()

plt.xlabel('Wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.tight_layout()

# plt.savefig('./figures/type_iax_comparison.pdf')

## Same SALT2 parameters but not at the same location in the manifold

In [ ]:
def outer_diff(x):
    return (x - x[:, None])

count = np.array([len(i.spectra) for i in a.targets])
first_phase = np.array([i.spectra[0].phase for i in a.targets])
mask = (count > 8) & (a.redshifts > 0.03) & (first_phase < -2)

salt_x1_diff = outer_diff(a.salt_x1)
salt_c_diff = outer_diff(a.salt_color)
salt_diff = outer_diff(a.salt_x1)**2 + 100 * outer_diff(a.salt_color)**2
manif_diff = outer_diff(a.embedding[:, 0])**2 + outer_diff(a.embedding[:, 1])**2 + outer_diff(a.embedding[:, 2])**2

phase_diff = outer_diff(a.salt_phases[a.center_mask])

np.fill_diagonal(salt_diff, np.nan)
np.fill_diagonal(manif_diff, np.nan)

salt_diff[~mask] = np.nan
salt_diff[:, ~mask] = np.nan
manif_diff[~mask] = np.nan
manif_diff[:, ~mask] = np.nan

In [ ]:
cut = (salt_x1_diff > 0) & (salt_x1_diff < 0.2) & (np.abs(salt_c_diff) < 0.02) & (manif_diff > 30) & (np.abs(phase_diff) < 1.0)
print("Num objects:  %s" % np.sum(cut))
salt_diff_idx = [i[2] for i in np.where(cut)]

t1 = a.targets[salt_diff_idx[0]]
t2 = a.targets[salt_diff_idx[1]]

print("target 1:     %s" % t1)
print("target 2:     %s" % t2)
print("redshifts:    %.4f, %.4f" % (a.redshifts[salt_diff_idx[0]], a.redshifts[salt_diff_idx[1]]))
print("x_1 values:   %+.3f, %+.3f" % (a.salt_x1[salt_diff_idx[0]], a.salt_x1[salt_diff_idx[1]]))
print("c values:     %+.3f, %+.3f" % (a.salt_color[salt_diff_idx[0]], a.salt_color[salt_diff_idx[1]]))
print("mags:         %.3f, %.3f" % (a.mags[salt_diff_idx[0]], a.mags[salt_diff_idx[1]]))
print("salt HRs:     %.3f, %.3f" % (a.salt_hr[salt_diff_idx[0]], a.salt_hr[salt_diff_idx[1]]))
print("corr mags:    %.3f, %.3f" % (a.corr_mags[salt_diff_idx[0]], a.corr_mags[salt_diff_idx[1]]))
print("embedding 1:  %s" % a.embedding[salt_diff_idx[0]])
print("embedding 2:  %s" % a.embedding[salt_diff_idx[1]])

salt_comparision_mb_diff = np.abs(
    a.salt_hr[salt_diff_idx[0]]
    - a.salt_hr[salt_diff_idx[1]]
)
dm_1 = frac_to_mag(t1.salt_fit['x0_err'] / t1.salt_fit['x0'])
dm_2 = frac_to_mag(t2.salt_fit['x0_err'] / t2.salt_fit['x0'])

salt_comparison_mb_diff_err = np.sqrt(dm_1**2 + dm_2**2)
print("mag_diff:     %.3f +/- %.3f" % (salt_comparision_mb_diff, salt_comparison_mb_diff_err))

In [ ]:
bands = ['U', 'B', 'V', 'R', 'I']

band_colors = {
    'U': 'C4',
    'B': 'C0',
    'V': 'C2',
    'R': 'C3',
    'I': 'C1',
}

gap = 1.

def plot_lightcurve(target, marker='o', ls='--', mfc='none', label_bands=True):
    photometry = target.get_photometry(bands, clip_filter=True)
    
    # Cut late phases
    photometry = photometry[photometry['time'] < 30]
        
    ref_mag = target.salt_fit['fitted_model'].source_peakmag('snfb', 'ab')
    
    for offset, band in enumerate(bands):
        band_photometry = photometry[photometry['band'] == 'snf%s' % band.lower()]
        
        if band == 'B':
            label = target.name
        else:
            label = ''
            
        color = band_colors[band]
        
        phases = band_photometry['time']
        mags = -2.5*np.log10(band_photometry['flux']) - ref_mag - (offset - 1) * gap
        magerrs = band_photometry['magerr']
    
        plt.errorbar(phases, mags, magerrs, fmt='none', c=color)
        plt.plot(phases, mags, label=label, marker=marker, c=color, mfc=mfc, ls=ls)
        
        if label_bands:
            plt.text(phases[-1] + 2, mags[-1] + 0.2, '%s%+d' % (band, offset - 1), c=color)
            plt.xlim(None, phases[-1] + 9)
            
def plot_salt_lightcurve(target, phase_min=-6, phase_max=50):
    model = target.salt_fit['fitted_model']
    phases = np.linspace(phase_min, phase_max, 200)
    
    # Compute the reference magnitude for B-band at max.
    ref_mag = model.source_peakmag('snfb', 'ab')

    # Plot each light curve
    for offset, band in enumerate(bands):
        mags = model.bandmag('snf%s' % band.lower(), 'ab', phases) - ref_mag - (offset - 1) * gap
        plt.plot(phases, mags, c='k')


plt.figure(figsize=(5, 5))
plot_lightcurve(t1, marker='^', ls='', mfc=None, label_bands=False)
plot_lightcurve(t2, marker='o', ls='', mfc='none', label_bands=True)
# plot_salt_lightcurve(t1)
plt.legend()
plt.xlabel('Phase (days)')
plt.ylabel('Normalized magnitude + offset')
plt.ylim(4.8, -2.5)
plt.tight_layout()

plt.savefig('./figures/same_salt_comparison.pdf')

# plt.gca().invert_yaxis()

In [ ]:
plt.figure()

def plot_spec(idx):
    spec = a.spectra[a.center_mask][idx]

    ref_mag = spec.target.salt_fit['fitted_model'].source_peakmag('snfb', 'ab')
    
    label = '%s, %.2f days' % (spec.target.name, spec.phase)
    
    scale = 10**(+0.4*ref_mag)
    
    plt.plot(a.wave, spec.flux * scale * spectrum_plot_scale, label=label)

plot_spec(salt_diff_idx[0])
plot_spec(salt_diff_idx[1])

plt.legend()

plt.xlabel('Restframe wavelength ($\AA$)')
plt.ylabel(spectrum_plot_ylabel)
plt.tight_layout()

plt.savefig('./figures/same_salt_spectra.pdf')

In [ ]:
# Dump details to latex
with open('latex/salt_comparison.tex', 'w') as f:
    latex_print(f, "")
    t1 = a.targets[salt_diff_idx[0]]
    t2 = a.targets[salt_diff_idx[1]]
    latex_command(f, 'saltcompnamea', '%s', t1.name)
    latex_command(f, 'saltcompnameb', '%s', t2.name)
    latex_command(f, 'saltcompxonea', '%.3f $\\pm$ %.3f', (t1['salt2.X1'], t1['salt2.X1.err']))
    latex_command(f, 'saltcompxoneb', '%.3f $\\pm$ %.3f', (t2['salt2.X1'], t2['salt2.X1.err']))
    latex_command(f, 'saltcompca', '%.3f $\\pm$ %.3f', (t1['salt2.Color'], t1['salt2.Color.err']))
    latex_command(f, 'saltcompcb', '%.3f $\\pm$ %.3f', (t2['salt2.Color'], t2['salt2.Color.err']))
    latex_command(f, 'saltcompcoorda', '%.2f', a.embedding[salt_diff_idx[0], 0])
    latex_command(f, 'saltcompcoordb', '%.2f', a.embedding[salt_diff_idx[1], 0])
    latex_command(f, 'saltcompmagbdiff', '%.3f $\\pm$ %.3f', (salt_comparision_mb_diff, salt_comparison_mb_diff_err))

## General variables

In [ ]:
# Define a bunch of functions to make things easier.
def latex_host_step(file, name, var, mags, mask):
    step, step_err = analyze_host_variable(var, mags, mask, plot=False)
    latex_command(file, name, '%.3f $\\pm$ %.3f', (np.abs(step), step_err))

a.fit_gp(kind='salt_raw', verbose=False)
salt_isomap_mags = a.corr_mags.copy()

a.fit_gp(verbose=False)
rbtl_isomap_mags = a.corr_mags.copy()

with open('latex/commands.tex', 'w') as f:
    latex_print(f, "")
    latex_command(f, 'numdatasetsne', '%d', len(a.dataset.targets))
    latex_command(f, 'numdatasetspectra', '%d', np.sum([len(i.spectra) for i in a.dataset.targets]))
    latex_print(f, "")
    latex_command(f, 'nummanifoldsne', '%d', len(a.targets))
    latex_command(f, 'nummanifoldspectra', '%d', len(a.spectra))
    latex_command(f, 'numinterpsne', '%d', np.sum(a.interp_mask))
    latex_print(f, "")
    latex_command(f, 'numsnftrain', '%d', np.sum([i.subset == 'training' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfvalid', '%d', np.sum([i.subset == 'validation' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfother', '%d', np.sum([i.subset not in ['training', 'validation'] for i in a.targets[a.interp_mask]]))
    latex_print(f, "")
    latex_command(f, 'numsnredshift', '%d', np.sum(a.interp_mask & (a.redshift_errs >= 0.004)))
    latex_command(f, 'numlowredshift', '%d', np.sum(a.interp_mask & (a.redshifts <= 0.02)))
    latex_command(f, 'numhighav', '%d', np.sum(a.interp_mask & (a.colors - np.nanmedian(a.colors) >= 0.5)))
    latex_print(f, "")
    latex_command(f, 'nummagsne', '%d', np.sum(a.interp_mask & a.redshift_color_mask))
    latex_command(f, 'nummagsnetrain', '%d', np.sum(a.good_mag_mask))
    latex_command(f, 'nummagsnevalidation', '%d', np.sum(a.interp_mask & a.redshift_color_mask & ~a.good_mag_mask))
    latex_print(f, "")
    latex_command(f, 'saltparammb', '%.2f', a.salt_MB)
    latex_command(f, 'saltparamalpha', '%.3f', a.salt_alpha)
    latex_command(f, 'saltparambeta', '%.2f', a.salt_beta)
    latex_command(f, 'saltparamsigmaint', '%.3f', a.salt_intrinsic_dispersion)
    # latex_command(f, 'saltparamrms', '%.3f', np.std(a.salt_hr[a.good_salt_mask]))
    latex_std(f, 'saltparamrms', a.salt_hr[a.good_salt_mask])
    latex_nmad(f, 'saltparamnmad', a.salt_hr[a.good_salt_mask])
    latex_command(f, 'saltparamwrms', '%.3f', a.salt_wrms)
    latex_command(f, 'saltparammindisp', '%.2f', np.min(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_command(f, 'saltparammaxdisp', '%.2f', np.max(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_print(f, "")
    latex_std(f, 'rawrbtlmagstd', a.mags[a.good_mag_mask])
    latex_nmad(f, 'rawrbtlmagnmad', a.mags[a.good_mag_mask])
    # latex_print(f, "")
    # latex_command(f, 'twinrbtlmagstd', '%.3f', a.twins_rms)
    # latex_command(f, 'twinrbtlmagnmad', '%.3f', a.twins_nmad)
    latex_print(f, "")
    latex_std(f, 'saltcomprawrbtlmagstd', a.mags[a.good_mag_mask & a.good_salt_mask])
    latex_std(f, 'saltcompsaltmagstd', a.salt_hr[a.good_mag_mask & a.good_salt_mask])

    a.fit_gp(verbose=False, kind='salt_raw')
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'saltgpcolor', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[0], gp_uncertainties[0]))
    latex_command(f, 'saltgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'saltgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'saltgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))
    latex_std(f, 'saltgprms', a.corr_mags[a.good_salt_mask & a.interp_mask])
    latex_std(f, 'saltgpcompsaltrms', a.salt_hr[a.good_salt_mask & a.interp_mask])

    a.fit_gp(verbose=False)
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'rbtlgpcolor', '%.2f $\\pm$ %.2f', (a.fiducial_rv * (1 + a.gp_hyperparameters[0]), a.fiducial_rv * gp_uncertainties[0]))
    latex_command(f, 'rbtlgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'rbtlgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'rbtlgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))

    latex_print(f, "")
    latex_std(f, 'rbtlgprms', a.corr_mags[a.good_mag_mask])
    latex_command(f, 'rbtlgpnmad', '%.3f', math.nmad(a.corr_mags[a.good_mag_mask]))

    latex_print(f, "")
    x1 = a.salt_hr[(a.embedding[:, 0] < 3) & a.good_salt_mask & a.interp_mask]
    x2 = a.salt_hr[(a.embedding[:, 0] > 3) & a.good_salt_mask & a.interp_mask]
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    err1 = np.std(x1) / np.sqrt(len(x1))
    err2 = np.std(x2) / np.sqrt(len(x2))
    latex_command(f, 'saltisomapdiff', '%.3f $\\pm$ %.3f', (np.abs(m1-m2), np.sqrt(err1**2 + err2**2)))

    latex_print(f, "")
    latex_command(f, 'pecvelcontribution', '%.3f', np.sqrt(np.mean(a.get_peculiar_velocity_uncertainty()[a.good_mag_mask & a.good_salt_mask]**2)))

    latex_print(f, "")
    latex_host_step(f, 'lssfrsaltxifull', 'lssfr', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'gmasssaltxifull', 'gmass', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'lssfrsaltxicut', 'lssfr', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltxicut', 'gmass', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrsaltisomapcut', 'lssfr', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltisomapcut', 'gmass', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrrbtlisomapcut', 'lssfr', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmassrbtlisomapcut', 'gmass', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_command(f, 'hostcutsnsnetrain', '%d', np.sum(a.good_mag_mask & a.good_salt_mask & a.host_mask))
    latex_command(f, 'hostcutsnsnefull', '%d', np.sum(a.redshift_color_mask & a.interp_mask & a.good_salt_mask & a.host_mask))

# Outliers

In [38]:
outliers_91t = [ 
    'SNF20070528-003', # Scalzo++ 2014
    'SNF20070803-005', # Scalzo++ 2014
    'SNF20070825-001', # Scalzo++ 2010
    'SNF20070912-000', # Scalzo++ 2014
    'SNF20080522-000', # Scalzo++ 2014
    'SNF20080723-012', # Scalzo++ 2014
    'SNF20080805-007', # Lin++ 2020
    'LSQ12cyz', # Lin++ 2020
    'LSQ12fhe', # Lin++ 2020
    'PTF11bju', # Lin++ 2020
    'PTF11mkx', # Lin++ 2020
]

outliers_91bg = [
    'LSQ12cfx', # Lin++ 2020
    'PTF10ops', # Maguire++ 2011
    'PTF11bkf', # Lin++ 2020
    'PTF11kjn', # Lin++ 2020
    'PTF11okh', # Lin++ 2020
    'PTF11pra', # Lin++ 2020
    'PTF12dwm', # Lin++ 2020
    'SN2005bl', # Lin++ 2020
    'SN2005dh', # Lin++ 2020 
    'SN2005dm', # Lin++ 2020
    'SN2007ba', # Lin++ 2020
    'SN2009hs', # Lin++ 2020
    'SNNGC6430', # Lin++ 2020
]

outliers_02cx = [
    'SN2005cc', # Lin++ 2020
]

mask_91t = np.array([i.name in outliers_91t for i in a.targets])
mask_91bg = np.array([i.name in outliers_91bg for i in a.targets])
mask_02cx = np.array([i.name in outliers_02cx for i in a.targets])

In [39]:
plt.figure()

use_x = a.embedding[:, 0]

mask = a.salt_mask & a.redshift_color_mask & a.uncertainty_mask
plt.errorbar(use_x[mask], a.salt_hr[mask], a.salt_hr_uncertainties[mask], label='Individual supernovae', fmt='.', alpha=0.2, c='k')
math.plot_binned_mean(use_x[mask], a.salt_hr[mask], c='C3', lw=2, label='Binned mean')

# mask = mask & mask_91t
# plt.scatter(use_x[mask], a.salt_hr[mask], label='91T-like SNe IA', c='C2', marker='s', zorder=10)

plt.legend()

plt.gca().invert_yaxis()

plt.xlabel('Component 1')
plt.ylabel('SALT2 Hubble residuals')

plt.tight_layout()

plt.savefig('/home/kyle/supernova/meetings/2020_01_04_aas_hawaii/talk/salt_bias.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
def do_plot(label, mask=None, **kwargs):
    if mask is None:
        mask = a.uncertainty_mask
    else:
        mask = mask & a.uncertainty_mask

    plt.scatter(a.embedding[mask, 0], a.embedding[mask, 1], label=label, s=50, **kwargs)
    
plt.figure()
do_plot('All SNe Ia', c='k', alpha=0.1)
do_plot('91T-like', mask_91t)
do_plot('91bg-like', mask_91bg)
do_plot('02cx-like', mask_02cx)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …